In [48]:
import re
import sys
import pandas as pd
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pycountry
from unidecode import unidecode

In [49]:
def column_data_type(df: pd.DataFrame) -> pd.DataFrame:
    """
    Retrieve the data types of columns in the DataFrame.
    Parameters:
        df (pd.DataFrame): The input DataFrame.
    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Column Name' and 'd_type'.
    """
    dtype_df = pd.DataFrame({'Column Name': df.columns, 'd_type': df.dtypes.values})
    dtype_df.reset_index(drop=True, inplace=True)

    return dtype_df

In [50]:
def get_null_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the null percentage for each column in the DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Column Name' and 'Null Percentage'.
    """
    null_percentage = (df.isnull().sum() / len(df)) * 100
    null_df = pd.DataFrame({'Column Name': null_percentage.index, 'Null Percentage': null_percentage.values})
    null_df.set_index('Column Name', inplace=True)
    null_df.index.name = None

    return null_df

In [51]:
def calculate_null_difference(df: pd.DataFrame, initial_null_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the difference between the initial null percentage and the actual null percentage for each column
    in the DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        initial_null_df (pd.DataFrame): DataFrame containing the initial null percentages for each column.

    Returns:
        pd.DataFrame: A DataFrame containing the columns 'Column', 'Initial Null Percentage',
        'Actual Null Percentage', and 'Difference'.
    """
    # Calculate the actual null percentages
    actual_null_percentage = (df.isnull().sum() / len(df)) * 100

    # Create a new DataFrame to store the differences
    difference_df = pd.DataFrame(columns=['Column', 'Initial Null Percentage', 'Actual Null Percentage', 'Difference'])

    # Iterate over the columns in initial_null_df
    for index, row in initial_null_df.iterrows():
        column = index
        initial_null_percentage = row['Null Percentage']

        # Check if the column exists in the actual null percentages DataFrame
        if column in actual_null_percentage.index:
            actual_percentage = round(actual_null_percentage[column],2)
            difference = actual_percentage - initial_null_percentage
        else:
            # Column does not exist in the actual null percentages DataFrame
            actual_percentage = 'Deleted'
            difference = 100.00

        # Create a DataFrame with the current difference row
        current_difference_df = pd.DataFrame({'Column': [column],
                                              'Initial Null Percentage': [round(initial_null_percentage,2)],
                                              'Actual Null Percentage': [actual_percentage],
                                              'Difference': [round(difference,2)]})

        # Concatenate the current difference row to the difference_df DataFrame
        difference_df = pd.concat([difference_df, current_difference_df], ignore_index=True)

    return difference_df

In [52]:
def count_null_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Counts the number of null values in each row of the DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame with two columns: 'Index' and 'Null Percentage'.
    """
    null_counts = df.isnull().sum(axis=1)
    null_percentage = (null_counts / df.shape[1]) * 100
    null_df = pd.DataFrame({'Index': df.index, 'Null Percentage': null_percentage})
    null_df = null_df.sort_values(by='Null Percentage', ascending=False).reset_index(drop=True)

    return null_df

In [53]:
def split_column_by_hyphen(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Splits a specified column, adds a hyphen, and joins it back together.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column to split and join.

    Returns:
        pd.DataFrame: The DataFrame with the updated column.
    """
    # Convert the specified column to string type, split each value, add a hyphen, and join them back together
    df[column] = df[column].astype(str).apply(lambda x: x[:2] + '-' + x[2:])
    
    return df

In [54]:
def update_location(df: pd.DataFrame, index: int) -> None:
    """
    Updates the location information for a single row based on latitude and longitude values.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the row to update.
        index (int): The index of the row to update.

    Returns:
        None
    """
    geolocator = Nominatim(user_agent="http")

    latitude = str(df.at[index, 'Latitude1'])
    longitude = str(df.at[index, 'Longitude1'])

    try:
        # Retrieve location information using geolocator
        location = geolocator.reverse(latitude + ',' + longitude, language='en')

        # Update the DataFrame if location information is available
        if location is not None and location.raw is not None:
            raw_data = location.raw.get('address', {})

            if 'display_name' in location.raw:
                df.at[index, 'Location'] = location.raw['display_name']

            if 'city' in raw_data:
                df.at[index, 'NAME'] = raw_data['state']

            if 'country' in raw_data:
                df.at[index, 'COUNTRY'] = raw_data['country']
    except (GeocoderTimedOut, KeyError, ValueError):
        # Handle specific geocoding errors
        pass


In [55]:
def fill_null_countries(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fills missing values in the 'Location' column by retrieving location information
    using latitude and longitude values for each country.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the updated 'Location', 'NAME', 'COUNTRY',
        'COUNTRYAFF', and 'ISO_CC' columns.
    """

    def echo(string, padding=80):
        padding = " " * (padding - len(string)) if padding else ""
        print(string + padding, end='\r')
    
    # Get a list of countries with null values in the 'Location' column
    country_list = df[df['Location'].isnull()]['COUNTRY'].unique()
    country_len = len(df[df['Location'].isnull()]['COUNTRY'])
    # Counter for tracking progress
    i = 0
    # Iterate over each country
    for country in country_list:
        # Get a subset of the DataFrame for the current country
        country_df = df[df['Location'].isnull() & (df['COUNTRY'] == country)]

        # Iterate over each row in the subset
        for index, row in country_df.iterrows():
            update_location(df, index)

            i += 1
            progress = (f"{i}/{country_len} {row['COUNTRY']}")
            echo(progress)
            
    # Get a subset of the DataFrame for rows with null values in both 'Location' and 'COUNTRY' columns
    null_df = df[df['Location'].isnull() & df['COUNTRY'].isnull()]
    null_len = len(df[df['Location'].isnull() & df['COUNTRY']])
    # Counter for tracking progress
    i = 0
    # Iterate over each row in the subset
    for index, row in null_df.iterrows():
        update_location(df, index)

        i += 1
        progress = (f"{i}/{null_len} {row['COUNTRY']}")
        echo(progress)
        
    # Get a list of countries with null values in the 'Location' column
    country_list = df[df['COUNTRY'].isnull(
    )]['CountryName_FromSource'].unique()
    country_len = len(df[df['COUNTRY'].isnull()]['CountryName_FromSource'])
    # Counter for tracking progress
    i = 0
    # Iterate over each country
    for country in country_list:
        # Get a subset of the DataFrame for rows with null values in both 'Location' and 'COUNTRY' columns
        country_df = df[(df['COUNTRY'].isnull()) & (
            df['CountryName_FromSource'] == country)]

        # Iterate over each row in the subset
        for index, row in country_df.iterrows():
            update_location(df, index)

            i += 1
            progress = (f"{i}/{country_len} {row['COUNTRY']}")
            echo(progress)
            
    return df

In [56]:
def update_countries(df: pd.DataFrame) -> pd.DataFrame:
    """
    Update missing values in the 'COUNTRY' and 'COUNTRYAFF' columns based on a country list.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_list (list): A list of column names to iterate over.

    Returns:
        pd.DataFrame: The DataFrame with updated 'COUNTRY' columns.
    """
    column_list = ['Location', 'CountryName_FromSource']

    country_list = set()

    # Collect unique country values from specified columns
    for column in column_list:
        country_list |= set(df[df['COUNTRY'].isnull()][column].unique())

    # Iterate over country values
    for country in country_list:
        # Iterate over specified columns
        for column in column_list:
            # Get subset of rows where 'COUNTRY' is null and 'column' matches the country value
            subset = df[(df['COUNTRY'].isnull()) & (df[column] == country)]

            # Iterate over subset rows
            for index, row in subset.iterrows():
                # Check for matching country name, alpha-3 code, or alpha-2 code
                for country_name in pycountry.countries:
                    country_lower = country.strip().lower()

                    if country_name.name.lower() in country_lower:
                        df.at[index, 'COUNTRY'] = country_name.name
                        break
                    if country_name.alpha_3 in country_lower:
                        df.at[index, 'COUNTRY'] = country_name.name
                        break
                    if country_name.alpha_2 in country_lower:
                        df.at[index, 'COUNTRY'] = country_name.name
                        break

    return df

In [57]:
def fill_country_code(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fills missing values in the 'ISO_CC' column based on the 'COUNTRY' column using pycountry.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with updated 'ISO_CC' column.
    """
    # Create a mapping of countries to country codes
    country_mapping = {
        country.name: country.alpha_2
        for country in pycountry.countries
    }

    # Iterate over rows with missing values in 'ISO_CC' column
    for index, row in df[df['ISO_CC'].isna() & ~df['COUNTRY'].isna()].iterrows():
        country = row['COUNTRY']

        # Check if country exists in the mapping
        if country in country_mapping:
            # Update the 'ISO_CC' column with the country code
            df.at[index, 'ISO_CC'] = country_mapping[country]

    return df

In [58]:
def fill_country_subdivision(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fills missing values in the 'NAME', and 'LAND_TYPE' columns based on the 'Location' column using pycountry.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with updated 'NAME', and 'LAND_TYPE' columns.
    """
    # Create a mapping of subdivision names to subdivision codes
    subdivision_mapping = {
        subdivision.name: subdivision.code
        for subdivision in pycountry.subdivisions
    }

    # Iterate over rows with missing values in 'NAME' column
    for index, row in df[df['NAME'].isnull()].iterrows():
        location = str(row['Location'])

        matching_subdivision = None
        subdivision_code = None
        
        # Look for exact matches of subdivision names in the 'Location' column
        for subdivision_name, subdivision_code in subdivision_mapping.items():
            if unidecode(subdivision_name.lower()) in unidecode(location.lower()):
                matching_subdivision = subdivision_name
                break

        if matching_subdivision:
            # Update the corresponding columns with the subdivision information
            df.at[index, 'NAME'] = pycountry.subdivisions.get(code=subdivision_code).name
            df.at[index, 'LAND_TYPE'] = pycountry.subdivisions.get(code=subdivision_code).type
        else:
            # Try to match subdivision using regular expressions on the 'Location' column
            for subdivision_name, subdivision_code in subdivision_mapping.items():
                if '-' in subdivision_code:
                    # Extract the second part of the subdivision code after the hyphen
                    escaped_subdivision_code = re.escape(subdivision_code.split('-')[1])
                else:
                    escaped_subdivision_code = re.escape(subdivision_code)
                    
                if re.search(r'\b' + escaped_subdivision_code + r'\b', location):
                    matching_subdivision = subdivision_name
                    
                    # Update the corresponding columns with the subdivision information
                    df.at[index, 'NAME'] = pycountry.subdivisions.get(code=subdivision_code).name
                    df.at[index, 'LAND_TYPE'] = pycountry.subdivisions.get(code=subdivision_code).type
                    break
    
    return df

In [59]:
df_location ='../data/raw/earth_challenge_dataset.csv'
df = pd.read_csv(df_location, parse_dates=['DateStandardized'])

/tmp/ipykernel_7584/407583611.py:2: DtypeWarning: Columns (5,21,22,23,24,34,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_location, parse_dates=['DateStandardized'])


In [60]:
null_df = get_null_percentage(df)
null_df.sort_values(by='Null Percentage',ascending=False)

,Null Percentage
TotalArea_Sq_m,100.000000
Other,100.000000
FieldObsevations,98.049202
BeachAreaLandcover,97.959109
BeachType,97.951754
DebrisDescription,97.935206
WaterfrontName,97.598735
TotalWidth_m,96.100243
StartTime,96.100243
Longitude2,91.654409


In [61]:
df = split_column_by_hyphen(df, 'ISO_CODE')

In [62]:

df = update_countries(df)

In [63]:
df = fill_null_countries(df)

In [64]:
# df = fill_country_code(df)

In [65]:
df = fill_country_subdivision(df)

In [66]:
df = df.drop(index=df[(df['COUNTRY'].isna())].index)
df = df.drop(index=df[df['Location'].isna()].index)

In [67]:
drop_list = ['TotalArea_Sq_m', 'Other', 'FieldObsevations', 'BeachAreaLandcover', 'BeachType', 'DebrisDescription', \
            'WaterfrontName', 'TotalWidth_m', 'StartTime', 'Longitude2', 'ShorelineName', 'Latitude2', 'X', 'Y', \
            'UniqueID', 'SourceID', 'SubCountry_L1_FromSource', 'SubCountry_L2_FromSource', 'CountryName_FromSource',\
            'OBJECTID', "ISO_SUB", "ISO_CC", "ADMINTYPE","RecordSequenceID", "DateOriginal", "MonthYear","Year", \
            "MonthNum", "Month", "Day", "DOW", "COUNTRYAFF", "DISPUTED", "NOTES", "AUTONOMOUS", "LocationFreqID"]

In [68]:
df = df.drop(drop_list, axis=1)

In [69]:
null_diff = calculate_null_difference(df, null_df)
null_diff.sort_values(by="Difference", ascending=True)

,Column,Initial Null Percentage,Actual Null Percentage,Difference
7,Location,5.97,0.0,-5.97
64,COUNTRY,1.24,0.0,-1.24
65,ISO_CODE,1.24,0.0,-1.24
63,NAME,1.24,0.15,-1.09
74,LAND_TYPE,1.24,0.3,-0.94
76,Shape__Area,1.24,0.97,-0.27
75,LAND_RANK,1.24,0.97,-0.27
73,CONTINENT,1.24,0.97,-0.27
77,Shape__Length,1.24,0.97,-0.27
26,TotalVolunteers,4.45,4.32,-0.13
